# COMP 448/548 – Medical Image Analysis
##            Homework #3

Eren Berke Demirbaş
64627
Batuhan Yalçın
64274

25.05.2022

[link text](https://colab.research.google.com/drive/1Jjk1ZASbFzSlDXXThrAROoL-ofsXtpq7?usp=sharing)

In [ ]:
from __future__ import print_function, division
import torch
import torch.nn as nn
import torch.optim as optim
from torch.optim import lr_scheduler
import numpy as np
import torchvision
from torchvision import datasets, models, transforms
import matplotlib.pyplot as plt
import time
import os
import copy
from google.colab import drive
from sklearn.metrics import confusion_matrix

In [ ]:
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
data_directory = '/content/drive/MyDrive/COMP448_HW3/main_dir/'
os.chdir(data_directory)

In [ ]:
data_transforms = {
 'train': transforms.Compose([
 # put the input to Tensor format in order to use torch
    transforms.Resize(224),
    transforms.ToTensor(), 
    #transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
 ]),
 'valid': transforms.Compose([
    transforms.Resize(224),
    transforms.ToTensor(),
    #transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
   ]),                        
 'test': transforms.Compose([
    transforms.Resize(224),
    transforms.ToTensor(),
    #transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])                     
   ])
}
image_datasets = {x: datasets.ImageFolder(os.path.join(data_directory, x),
 data_transforms[x]) for x in ['train', 'valid', 'test']}

In [ ]:
def determine_weights_of_balanced_classes(images, K):
    counter = list(0 for i in range(0, K))
    balanced_weights = {}
    for type in ['train', 'valid', 'test']:
        for element in images[type]:
            counter[element[1]] = counter[element[1]] + 1
        class_weights = list(0.0 for i in range(0,K))
        for c in range(K):
            class_weights[c] = sum(counter) / counter[c]
        #print(len(images[type]))
        weight = list(0 for i in range(0, len(images[type])))
        for index, value in enumerate(images[type]):
            weight[index] = class_weights[value[1]]
        balanced_weights[type] = weight
    return balanced_weights  

  

In [ ]:
balanced_weights = determine_weights_of_balanced_classes(image_datasets, 3)
print(balanced_weights)
sampler = {x: torch.utils.data.sampler.WeightedRandomSampler(balanced_weights[x], len(balanced_weights[x])) for x in ['train', 'valid', 'test']}
dataloaders = {x: torch.utils.data.DataLoader(image_datasets[x],batch_size = 4, shuffle = True, num_workers = 4) for x in ['train', 'valid', 'test']}

{'train': [3.26, 3.26, 3.26, 3.26, 3.26, 3.26, 3.26, 3.26, 3.26, 3.26, 3.26, 3.26, 3.26, 3.26, 3.26, 3.26, 3.26, 3.26, 3.26, 3.26, 3.26, 3.26, 3.26, 3.26, 3.26, 3.26, 3.26, 3.26, 3.26, 3.26, 3.26, 3.26, 3.26, 3.26, 3.26, 3.26, 3.26, 3.26, 3.26, 3.26, 3.26, 3.26, 3.26, 3.26, 3.26, 3.26, 3.26, 3.26, 3.26, 3.26, 2.0632911392405062, 2.0632911392405062, 2.0632911392405062, 2.0632911392405062, 2.0632911392405062, 2.0632911392405062, 2.0632911392405062, 2.0632911392405062, 2.0632911392405062, 2.0632911392405062, 2.0632911392405062, 2.0632911392405062, 2.0632911392405062, 2.0632911392405062, 2.0632911392405062, 2.0632911392405062, 2.0632911392405062, 2.0632911392405062, 2.0632911392405062, 2.0632911392405062, 2.0632911392405062, 2.0632911392405062, 2.0632911392405062, 2.0632911392405062, 2.0632911392405062, 2.0632911392405062, 2.0632911392405062, 2.0632911392405062, 2.0632911392405062, 2.0632911392405062, 2.0632911392405062, 2.0632911392405062, 2.0632911392405062, 2.0632911392405062, 2.0632911

/usr/local/lib/python3.7/dist-packages/torch/utils/data/dataloader.py:490: UserWarning: This DataLoader will create 4 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  cpuset_checked))


In [ ]:
def train_model(model, criterion, optimizer,
 scheduler, num_epochs):
    best_model_wts = copy.deepcopy(model.state_dict())
    best_no_corrects = 0
    for epoch in range(num_epochs):
        # Set the model to the training mode for updating the weights using
        # the first portion of training images
        model.train()
        for inputs, labels in dataloaders['train']: # iterate over data
            inputs = inputs.to(device)
            labels = labels.to(device)
            optimizer.zero_grad()
            with torch.set_grad_enabled(True):
                outputs = model(inputs)
                _, preds = torch.max(outputs, 1)
                loss = criterion(outputs, labels)
                loss.backward()
                optimizer.step()
        # Set the model to the evaluation mode for selecting the best network
        # based on the number of correctly classified validation images
        model.eval()
        no_corrects = 0
        for inputs, labels in dataloaders['valid']:
            inputs = inputs.to(device)
            labels = labels.to(device)
            with torch.set_grad_enabled(False):
                outputs = model(inputs)
                _, preds = torch.max(outputs, 1)
                no_corrects += torch.sum(preds == labels.data)
            if no_corrects > best_no_corrects:
                best_no_corrects = no_corrects
                best_model_wts = copy.deepcopy(model.state_dict())
            scheduler.step()
    # Load the weights of the best network
    model.load_state_dict(best_model_wts)
    return model

In [ ]:
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
model_conv = models.alexnet(pretrained = True)
for param in model_conv.parameters():
    param.requires_grad = False

In [ ]:
features = model_conv.classifier[6].in_features # applies a linear transformation to the incoming data
print(features)
model_conv.classifier[6] = nn.Linear(features, 3)
model_conv = model_conv.to(device) # use the GPU

4096


In [ ]:
criterion = nn.CrossEntropyLoss() #Computes the entropy loss between input and target
optimizer_conv = optim.SGD(model_conv.parameters(), lr=0.001, momentum=0.9)
exp_lr_scheduler = lr_scheduler.StepLR(optimizer_conv, step_size=1, gamma=0.1)
model_conv.eval

<bound method Module.eval of AlexNet(
  (features): Sequential(
    (0): Conv2d(3, 64, kernel_size=(11, 11), stride=(4, 4), padding=(2, 2))
    (1): ReLU(inplace=True)
    (2): MaxPool2d(kernel_size=3, stride=2, padding=0, dilation=1, ceil_mode=False)
    (3): Conv2d(64, 192, kernel_size=(5, 5), stride=(1, 1), padding=(2, 2))
    (4): ReLU(inplace=True)
    (5): MaxPool2d(kernel_size=3, stride=2, padding=0, dilation=1, ceil_mode=False)
    (6): Conv2d(192, 384, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (7): ReLU(inplace=True)
    (8): Conv2d(384, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (9): ReLU(inplace=True)
    (10): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (11): ReLU(inplace=True)
    (12): MaxPool2d(kernel_size=3, stride=2, padding=0, dilation=1, ceil_mode=False)
  )
  (avgpool): AdaptiveAvgPool2d(output_size=(6, 6))
  (classifier): Sequential(
    (0): Dropout(p=0.5, inplace=False)
    (1): Linear(in_features=9216, ou

In [ ]:
model_conv = train_model(model_conv, criterion, optimizer_conv,exp_lr_scheduler, num_epochs=25)

/usr/local/lib/python3.7/dist-packages/torch/utils/data/dataloader.py:490: UserWarning: This DataLoader will create 4 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  cpuset_checked))


In [ ]:
from torchvision.transforms.functional import normalize
#Pytorch confusion matrix
confusion_matrix = torch.zeros(3, 3)
with torch.no_grad():
  for case in ['train', 'valid', 'test']:
      for i, (inputs, classes) in enumerate(dataloaders[case]):
        inputs = inputs.to(device)
        classes = classes.to(device)
        outputs = model_conv(inputs)
        tests, preds = torch.max(outputs, 1)
        for labels, predict in zip(classes.view(-1), preds.view(-1)):
          confusion_matrix[labels.long(), predict.long()] += 1

      print("Confusion Matrix for ",case ,"is:\n",confusion_matrix)
      #Vectorize Confusion matrix and get class accuracies
      Class_Accuracy=confusion_matrix.diagonal()/(confusion_matrix.sum(0) - confusion_matrix.diagonal() + confusion_matrix.sum(1) )
      print("Class Accuracies are",Class_Accuracy)
      #Overall accuracy
      Overall_accuracy = confusion_matrix.diagonal().sum()/confusion_matrix.sum()
      print("Overall_accuracy is :",Overall_accuracy)


/usr/local/lib/python3.7/dist-packages/torch/utils/data/dataloader.py:490: UserWarning: This DataLoader will create 4 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  cpuset_checked))


Confusion Matrix for  train is:
 tensor([[49.,  1.,  0.],
        [ 8., 70.,  1.],
        [ 9.,  3., 22.]])
Class Accuracies are tensor([0.7313, 0.8434, 0.6286])
Overall_accuracy is : tensor(0.8650)
Confusion Matrix for  valid is:
 tensor([[59.,  1.,  0.],
        [10., 77.,  1.],
        [10.,  4., 24.]])
Class Accuracies are tensor([0.7375, 0.8280, 0.6154])
Overall_accuracy is : tensor(0.8602)
Confusion Matrix for  test is:
 tensor([[104.,   4.,   0.],
        [ 13., 127.,   5.],
        [ 21.,   7.,  49.]])
Class Accuracies are tensor([0.7324, 0.8141, 0.5976])
Overall_accuracy is : tensor(0.8485)
